# PNOFi

![DoNOF](https://donof.readthedocs.io/en/latest/_images/Logo-DoNOF.jpeg)

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

En este notebook se encuentra el álgebra elemental para calcular un punto simple de energía de PNOFi (i=5,6,7)

**Seleccionamos un PNOFi (i=5,7)** y el tipo de gradiente, $\frac{dE}{dn_i}$, para los n.umeros de ocupación.

In [47]:
PNOFi = 7
gradient = "analytical" # analytical/numerical

## Introducción

**Importamos librerías**

In [48]:
import psi4
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import eigh
from time import time

**Seleccionamos una molécula**, y otros datos como la memoria del sistema y la base

In [49]:
psi4.set_memory('4 GB')

mol = psi4.geometry("""
O  0.0000   0.000   0.116
H  0.0000   0.749  -0.453
H  0.0000  -0.749  -0.453
  symmetry c1
""")

psi4.set_options({'basis': 'cc-pVDZ',
                  'scf_type': 'pk',
                  'e_convergence': 1e-8})

Construimos la función de onda y **evaluamos matrices e integrales en orbital atómico**, $S$, $T$, $V$, $H$, $(\mu\nu|\sigma\lambda)$

In [52]:
# Wavefunction
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis'))

# Integrador
mints = psi4.core.MintsHelper(wfn.basisset())

# Overlap, Kinetics, Potential
S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
H = T + V

# Integrales de Repulsión Electrónica, ERIs (mu nu | sigma lambda)
I = np.asarray(mints.ao_eri())

# Energía Nuclear
E_nuc = mol.nuclear_repulsion_energy()

**Se declaran variables del sistema.**

El número de electrones (ne) es la suma de los electrones con spín alpha (nalpha) y los electrones con spín beta (nbeta)
\begin{equation}
N_e = N_{\alpha} + N_{\beta}
\end{equation}

El número de orbitales doblemente ocupados inactivos (no1), es decir con ocupación de 1, está dado por
\begin{equation}
N_{o1} = \left\{
  \begin{array}{lll}
  \sum_{átomo}^{N_{átomos}} {orbitales\ de\ core}_{átomo}      & Default \\
  Otro     & \mathrm{si\ se\ indica}
  \end{array}
  \right.
\end{equation}

*****************************************************************************************************************

Dividiremos el espacio orbital en dos subespacios
\begin{equation}
\Omega = \Omega_I + \Omega_{II}
\end{equation}

*****************************************************************************************************************

El subespacio $\Omega_{II}$ se divide en $N_{II}/2$ subespacios $\Omega_g$. Cada subespacio $\Omega_g \in \Omega_{II}$ contiene un orbital $\ket{g}$ con $g \le N_{II}/2$ y $N_g$ orbitales $\ket{p}$ con $p > N_{II}/2$, es decir
\begin{equation}
\Omega_g = \{\ket{g},\ket{p_1},\ket{p_2},\cdots,\ket{p_{N_{g}}}\} 
\end{equation}

El número de orbitales fuertemente doble ocupados (ndoc) está dado por
\begin{equation}
N_{doc} = N_{\beta} - N_{o1} = N_{II}/2
\end{equation}

*****************************************************************************************************************

El subespacio $\Omega_I$ se compone por $N_I$ subespacios $\Omega_g$, en este caso

El número de orbitales fuertemente ocupados con ocupación simple (nsoc) está dado por
\begin{equation}
N_{soc} = N_{\alpha} - N_{\beta} = N_I
\end{equation}

*****************************************************************************************************************

El número de orbitales fuertemente ocupados (ndns) está dado por
\begin{equation}
N_{ndns} = N_{doc} + N_{soc} = N_{II}/2 + N_I = N_{\Omega}
\end{equation}

El número de orbitales virtuales (nvir) es la diferencia entre los electrones $N_\alpha$ (nalpha) y el número de funciones de base $N_{bf}$ 
\begin{equation}
N_{vir} = N_{bf} - N_{\alpha}
\end{equation}

El número de orbitales débilmente ocupados acoplados (ncwo) a cada orbital fuertemente doble ocupado (ndoc), y que constituye $N_g$, está dado por
\begin{equation}
N_{cwo} = \left\{
  \begin{array}{lll}
  N_{vir}/N_{doc}      & \mathrm{si\ } N_e = 2 \\
  1 & \mathrm{si\ } N_e > 2 \mathrm{\ ó\ } N_{cwo}>N_{vir}/N_{doc}\\
  Otro     & \mathrm{si\ se\ indica}
  \end{array}
  \right.
\end{equation}

*****************************************************************************************************************

La dimensión del subespacio de orbitales activos está dada por los orbitales fuertemente doble ocupados y sus orbitales débilmente ocupados asociados
\begin{equation}
N_{ac} = N_{doc} + N_{doc} N_{cwo}
\end{equation}

*****************************************************************************************************************

Los orbitals con número de ocupación distinto de cero (nbf5) son
\begin{equation}
N_{bf5} = N_{o1} + N_{ac} + N_{soc} 
\end{equation}
es decir, la suma de los orbitales de core (no1),más los fuertemente doble ocupados (ndoc) con sus orbitales débilmente ocupados acoplados (nwco) y los orbitales fuertemente ocupados con ocupación simple (nsoc)
\begin{equation}
N_{bf5} = N_{o1} + N_{doc} + N_{doc}N_{wco} + N_{soc}
\end{equation}

*****************************************************************************************************************

Establecemos los orbitales a optimizar (noptorb)
\begin{equation}
N_{optorb} = \left\{
  \begin{array}{lll}
  N_{bf}      & Default \\
  Otro     & \mathrm{si\ se\ indica}
  \end{array}
  \right.
\end{equation}

In [12]:
natoms = mol.natom()
nbf = S.shape[0]
nalpha = wfn.nalpha()
nbeta = wfn.nbeta()
ne = nalpha + nbeta
mul = mol.multiplicity()
no1 = 0 #Number of inactive doubly occupied orbitals | Se puede variar
for i in range(natoms):
    Z = mol.Z(i)
    if ( 1<=Z and Z<=  2):
        no1 += 0           # H-He
    elif ( 3<=Z and Z<= 10):
        no1 +=  1          # Li-Ne
    elif (11<=Z and Z<= 18):
        no1 +=  5          # Na-Ar
    elif(19<=Z and Z<= 36):
        no1 +=  9          # K-Kr
    elif(37<=Z and Z<= 49):
        no1 += 18          # Rb-In
    elif(50<=Z and Z<= 54):
        no1 += 23          # Sn-Xe
    elif(55<=Z and Z<= 71):
        no1 += 27          # Cs-Lu
    elif(72<=Z and Z<= 81):
        no1 += 30          # Hf-Tl
    elif(82<=Z and Z<= 86):
        no1 += 39          # Pb-Rn
    elif(87<=Z and Z<=109):
        no1 += 43          # Fr-Mt
    
ndoc = nbeta   -   no1
nsoc = nalpha  -   nbeta
ndns = ndoc    +   nsoc
nvir = nbf     -   nalpha

ncwo = 1
if(ne==2):
    ncwo= -1
if(ndns!=0):
    if(ndoc>0):
        if(ncwo!=1):
            if(ncwo==-1 or ncwo > nvir/ndoc):
                ncwo = int(nvir/ndoc)
    else:
        ncwo = 0

nac = ndoc * (1 + ncwo)
nbf5 = no1 + nac + nsoc   #JFHLY warning: nbf must be >nbf5
no0 = nbf - nbf5

noptorb = nbf     

closed = (nbeta == (ne+mul-1)/2 and nalpha == (ne-mul+1)/2)

Establecemos algunos parámetros

In [17]:
maxit = 1000  # Número máximo de iteraciones de Occ-SCF
no1 = no1     # Número de orbitales inactivos con ocupación 1
thresheid = 10**-6#8 # Convergencia de la energía total
maxitid = 30  # Número máximo de iteraciones externas
ipnof = 7     # PNOFi a calcular
threshl = 10**-3#4   # Convergencia de los multiplicadores de Lagrange
threshe = 10**-4#6   # Convergencia de la energía
threshec = 10**-10 # Convergencia  de la energía en optimización orbital
threshen = 10**-10 # Convergencia  de la energía en optimización de ocupaciones
scaling = True     # Scaling for f
nzeros = 1#0
nzerosm = 5
nzerosr = 2
itziter = 10        # Iteraciones para scaling constante
diis = True         # DIIS en optimización orbital
thdiis = 10**-3     # Para iniciar DIIS
ndiis = 5           # Número de ciclos para interpolar matriz de Fock generalizada en DIIS
perdiis = True      # Aplica DIIS cada NDIIS (True) o después de NDIIS (False) 
ncwo = ncwo         # Número de orbitales débilmente ocupados acoplados a cada orbital fueremtente ocupado 
noptorb = noptorb   # Número de orbitales a optimizar Nbf5 <= Noptorb <= Nbf
scaling = True
maxloop = 30

Obtenemos un Guess acorde a la ecuación

\begin{equation}
HC = SC\varepsilon
\end{equation}

In [18]:
E_i,C = eigh(H, S)

**Revisamos la ortonormalidad.** Asumiremos que los orbitales son ortonormales si $C^T SC$ difiere en menos de $10^{-6}$ respecto a la matriz identidad. Si los orbitales no son ortonormales, haremos hasta 3 intentos por ortonormalizarlos.

In [19]:
# Revisa ortonormalidad
orthonormality = True

CTSC = np.matmul(np.matmul(np.transpose(C),S),C)
ortho_deviation = np.abs(CTSC - np.identity(nbf))

if (np.any(ortho_deviation > 10**-6)):
    orthonormality = False

if not orthonormality:
    print("Orthonormality violations {:d}, Maximum Violation {:f}".format((ortho_deviation > 10**-6).sum(),ortho_deviation.max()))        
else:
    print("No violations of the orthonormality")

No violations of the orthonormality


Se cambian de fase los obitales moleculares tal que **en cada orbital sea positivo el coeficiente de mayor magnitud**

In [20]:
# Vuelve positivo el elemento más largo de cada MO
for j in range(nbf):
    
    #Obtiene el índice del coeficiente con mayor valor absoluto del MO
    idxmaxabsval = 0
    for i in range(nbf):
        if(abs(C[i][j])>abs(C[idxmaxabsval][j])):
            idxmaxabsval = i
    
    # Ajusta el signo del MO
    sign = np.sign(C[idxmaxabsval][j])
    C[0:nbf,j] = sign*C[0:nbf,j]

## Cálculo de orbitales y multiplicadores de Lagrange a partir de J, K y F

Crearemos una función que calcule

\begin{eqnarray}
D_{\mu\nu}^{(i)} &=& C_{\mu i} C_{\nu i}\\
J_{\mu\nu}^{(i)} &=& \sum_{\sigma\lambda} D_{\sigma\lambda}^{(i)} (\mu\nu|\sigma\lambda)\\
K_{\mu\sigma}^{(i)} &=& \sum_{\nu\lambda} D_{\nu\lambda}^{(i)} (\mu\nu|\sigma\lambda)
\end{eqnarray}

In [21]:
def computeJK(C):
    
    #denmatj    
    D = np.einsum('mi,ni->imn', C[:,0:nbf5], C[:,0:nbf5], optimize=True)
    
    #hstarj
    J = np.einsum('isl,mnsl->imn', D, I, optimize=True)    
    
    #hstark
    K = np.einsum('inl,mnsl->ims', D, I, optimize=True)    

    return J,K

Construimos

\begin{equation}
\lambda_{qp} = n_p H_{qp} + \int dr \frac{d V_{ee}}{d \phi_p (r)} \phi_q (r)
\end{equation}

Definimos una función para calcular la matriz generalizada de Fock
\begin{equation}
F_{\mu\nu}^{(i)} = \left\{
  \begin{array}{lll}
  H_{\mu\nu} + \sum_j^{N_{bf5}} J_{\mu\nu}^{(j)} C^{J}_{1j} - \sum_j^{N_{bf5}} K_{\mu\nu}^{(j)} C^{K}_{1j}      & \mathrm{si\ } i \in [1,N_{o1}]  \\
  n_{i}(H_{\mu\nu} + J_{\mu\nu}^{(i)}) + \sum_{j \ne i}^{N_{bf5}} J_{\mu\nu}^{(j)} C^{J}_{ij} - \sum_{j \ne i}^{N_{bf5}} K_{\mu\nu}^{(j)} C^{K}_{ij}     & \mathrm{si\ } i \in (N_{o1},N_{beta}]\\
  n_{i}H_{\mu\nu} + \sum_{j \ne i}^{N_{bf5}} J_{\mu\nu}^{(j)} C^{J}_{ij} - \sum_{j \ne i}^{N_{bf5}} K_{\mu\nu}^{(j)} C^{K}_{ij}     & \mathrm{si\ } i \in (N_{beta},N_{alpha}]\\
  n_i(H_{\mu\nu} + J_{\mu\nu}^{(i)}) + \sum_{j \ne i}^{N_{bf5}} J_{\mu\nu}^{(j)} C^J_{ij} - \sum_{j \ne i}^{N_{bf5}} K_{\mu\nu}^{(j)} C^K_{ij}     & \mathrm{si\ } i \in (N_{\alpha},N_{bf5}]
  \end{array}
  \right.
\end{equation}

In [22]:
def computeF(J,K,n,cj12,ck12):
    
    # Matriz de Fock Generalizada                    
    F = np.zeros((nbf5,nbf,nbf))    

    ini = 0
    if(no1>1):        
        ini = no1       
        
    # nH
    F += np.einsum('i,mn->imn',n,H,optimize=True)        # i = [1,nbf5]

    # nJ
    F[ini:nbeta,:,:] += np.einsum('i,imn->imn',n[ini:nbeta],J[ini:nbeta,:,:],optimize=True)        # i = [ini,nbeta]
    F[nalpha:nbf5,:,:] += np.einsum('i,imn->imn',n[nalpha:nbf5],J[nalpha:nbf5,:,:],optimize=True)  # i = [nalpha,nbf5]
          
    # C^J J
    F += np.einsum('ij,jmn->imn',cj12,J,optimize=True)                                                # i = [1,nbf5]
    F[ini:nbf5,:,:] -= np.einsum('ii,imn->imn',cj12[ini:nbf5,ini:nbf5],J[ini:nbf5,:,:],optimize=True) # quita i==j

    # -C^K K
    F -= np.einsum('ij,jmn->imn',ck12,K,optimize=True)                                                # i = [1,nbf5]
    F[ini:nbf5,:,:] += np.einsum('ii,imn->imn',ck12[ini:nbf5,ini:nbf5],K[ini:nbf5,:,:],optimize=True) # quita i==j
      
    return F

Definimos una función que reciba los orbitales moleculares y la matriz de Fock y calcule los multiplicadores de lagrange según
Generamos 
\begin{equation}
G_{\mu}^{(i)} = \sum_{\nu}^{N_{bf}} F_{\mu\nu}^{(i)} C_{\nu i}
\end{equation}

\begin{equation}
\lambda_{ij} = \sum_{\mu}^{N_{bf}} C_{\mu i} G_{\mu j}
\end{equation}

In [23]:
def computeLagrange(F,C):
    
    G = np.einsum('imn,ni->mi',F,C[:,0:nbf5],optimize=True)
            
    #Compute Lagrange multipliers
    elag = np.zeros((nbf,nbf))
    elag[0:noptorb,0:nbf5] = np.einsum('mi,mj->ij',C[:,0:noptorb],G,optimize=True)[0:noptorb,0:nbf5]
                
    return elag

Definimos una función para calcular la energía electrónica
\begin{equation}
E = \sum_{i=1}^{N_{\beta}} \left( \lambda_{ii} + n_i \sum_{\mu\nu} C_{\mu i}H_{\mu\nu}C_{\nu i} \right) + \sum_{i=N_{\beta}+1}^{N_{\alpha}} \left( \lambda_{ii} + n_i \sum_{\mu\nu} C_{\mu i}H_{\mu\nu}C_{\nu i} \right) + \sum_{i=N_{\alpha+1}}^{N_{bf5}} \left( \lambda_{ii} + n_i \sum_{\mu\nu} C_{\mu i}H_{\mu\nu}C_{\nu i} \right)
\end{equation}

In [24]:
def computeE_elec(H,C,n,elag):
    #EELECTRr
    E = 0

    E = E + np.einsum('ii',elag[:nbf5,:nbf5],optimize=True)
    E = E + np.einsum('i,mi,mn,ni',n[:nbf5],C[:,:nbf5],H,C[:,:nbf5],optimize=True)
          
    return E

Definimos una función que calcule la convergencia de los multiplicadores de lagrange

In [25]:
def computeLagrangeConvergency(elag):
    # Convergency
    
    sumdiff = np.sum(np.abs(elag-elag.T))
    maxdiff = np.max(np.abs(elag-elag.T))

    return sumdiff,maxdiff

Finalmente definimos una función que reciba coeficientes y haga lo siguiente
1. Calcule $J_{\mu\nu}^{(i)}$ y $K_{\mu\nu}^{(i)}$
2. Calcule la matriz $F_{\mu\nu}^{(i)}$
3. Calcule los multiplicadores de Lagrange $\lambda_{ij}$
4. Calcule la energía
5. Revise la convergencia de los multiplicadores de Lagrange

In [26]:
def ENERGY1r(C,n,cj12,ck12):
    
    J,K = computeJK(C)

    F = computeF(J,K,n,cj12,ck12)

    elag = computeLagrange(F,C)

    E = computeE_elec(H,C,n,elag)

    sumdiff,maxdiff = computeLagrangeConvergency(elag)        
        
    return E,elag,sumdiff,maxdiff

## Iterative Diagonalizator Hartree-Fock

Definimos valores iniciales de números de ocupación (n).

\begin{equation}
n_{i} = \left\{
  \begin{array}{lll}
  1     & \mathrm{si\ } i \in [1,N_{\beta}]  \\
  0.5   & \mathrm{si\ } i \in [N_{\beta},N_{\alpha}]  \\
  0     & \mathrm{si\ otro\ caso} 
  \end{array}
  \right.
\end{equation}

Los funcionales PNOF son del tipo
\begin{equation}
E = C^{J}_{ij} J_{MO} + C^{K}_{ij} K_{MO}
\end{equation}

Donde $C^{J}_{ij}$ y $C^{K}_{ij}$ son ciertos coeficientes que se determinan para cada funcional y cumplen con ciertas reglas.

Definiremos valores iniciales para un HF
\begin{eqnarray}
C^J_{ij} &=& 2n_in_j\\
C^K_{ij} &=& n_in_j
\end{eqnarray}

In [27]:
no1_ori = no1
no1 = nbeta

n = np.zeros((nbf5))
n[0:nbeta] = 1.0
n[nbeta:nalpha] = 0.5

cj12 = 2*np.einsum('i,j->ij',n,n)
ck12 = np.einsum('i,j->ij',n,n)

Definimos una función que reciba los multiplicadores de Lagrange y calcule Fmiug

En la primera iteración
\begin{equation}
Fmiug_{ij} = \frac{\lambda_{ij}+\lambda_{ji}}{2}
\end{equation}

En las demás
\begin{equation}
Fmiug_{ij} =  \theta(j-i) (\lambda_{ij}-\lambda_{ji}) + \theta(i-j) (\lambda_{ji}-\lambda_{ij})
\end{equation}

Donde $\theta(x)$ es la función de Heaviside

Además, se aplica una técnica de escalamiento a los elementos fuera de la diagonal, tal que
\begin{equation}
Fmiug_{ij} = 
  \begin{array}{lll}
  0.1Fmiug_{ij}     & \mathrm{si\ } 10^{N_{zeros}+9-k} < |Fmiug_{ij}| < 10^{N_{zeros}+10-k}  \\
  \end{array}
\end{equation}
con $k \in [0,N_{zeros}+9]$

In [28]:
def fmiug_scaling(fmiug0,elag,i_ext,nzeros):
    
    #scaling
    fmiug = np.zeros((nbf,nbf))
    if(i_ext == 0):
        fmiug[:noptorb,:noptorb] = ((elag[:noptorb,:noptorb] + elag[:noptorb,:noptorb].T) / 2)

    else:
        fmiug[:noptorb,:noptorb] = (elag[:noptorb,:noptorb] - elag[:noptorb,:noptorb].T)
        fmiug = np.tril(fmiug,-1) + np.tril(fmiug,-1).T
        for k in range(nzeros+9+1):
            fmiug[(abs(fmiug) > 10**(9-k)) & (abs(fmiug) < 10**(10-k))] *= 0.1
        np.fill_diagonal(fmiug[:noptorb,:noptorb],fmiug0[:noptorb])

    return fmiug

Creamos el SCF del HFIDr, el cual consiste en iteraciones externas e internas. En cada iteración interna se sigue el siguiente procedimiento:
1. Calcular $Fmiug$ a partir de los multiplicadores de lagrange $\varepsilon$, aplicando escalamiento de ser necesario
2. Diagonalizar $Fmiug$
3. Obtener nuevos coeficientes de orbitales moleculares
4. Calcular nueva matriz generalizada de Fock $F_{\mu\nu}^{(i)}$, así como nuevos multiplicadores de Lagrange $\varepsilon_{ij}$
5. Revisar convergencia

In [29]:
#HFIDr

print('{:^7} {:^7} {:^14} {:^14} {:^15} {:^14}'.format("Nitext","Nitint","Eelec","Etot","Ediff","maxdiff"))

E,elag,sumdiff,maxdiff = ENERGY1r(C,n,cj12,ck12)

fmiug0 = np.zeros((nbf))
ext = True
# iteraciones externas
for i_ext in range(maxitid):
    if i_ext==0:
        maxlp = 1
    else:
        maxlp = maxloop
            
    # iteraciones internas             
    for i_int in range(maxlp):
        E_old = E

        if(scaling):
            fmiug = fmiug_scaling(fmiug0,elag,i_ext,nzeros)
                   
        fmiug0, W = np.linalg.eigh(fmiug)
        C = np.matmul(C,W)
        E,elag,sumdiff,maxdiff = ENERGY1r(C,n,cj12,ck12)

        E_diff = E-E_old                    
        if(abs(E_diff)<thresheid):
            print('{:6d} {:6d} {:14.8f} {:14.8f} {:14.8f} {:14.8f}'.format(i_ext,i_int,E,E+E_nuc,E_diff,maxdiff))
            for i in range(nbf):
                fmiug0[i] = elag[i][i]
            ext = False
            break
    if(not ext):
        break
    print('{:6d} {:6d} {:14.8f} {:14.8f} {:14.8f} {:14.8f}'.format(i_ext,i_int,E,E+E_nuc,E_diff,maxdiff))

Nitext  Nitint      Eelec           Etot           Ediff         maxdiff    
     0      0   -71.66636066   -62.31174996     5.45167028     1.82522751
     1     29   -85.21544417   -75.86083347    -0.03469315     0.21879491
     2     29   -85.36234639   -76.00773569     0.00254138     0.09306746
     3     29   -85.37481556   -76.02020486    -0.00098278     0.05864438
     4     29   -85.38169791   -76.02708721    -0.00002715     0.01154111
     5     26   -85.38189755   -76.02728685    -0.00000099     0.00194318


Regresamos no1 a su estado original

In [30]:
no1 = no1_ori

## PNOF

A continuación generamos las $C_{ij}^{J}$ y $C_{ij}^{K}$ que definen a los funcionales PNOF, así como las derivadas para los números de ocupación. 

**Nota.** En $\frac{d C_{ij}^J}{d\gamma_k}$ y $\frac{d C_{ij}^K}{d\gamma_k}$ se aprovecha la simetría y solo se toma la mitad de la derivada, al calcular el gradiente se multiplicará por 2.

Para PNOF5

\begin{equation}
E^{PNOF5} = \sum_{g=1}^{N_{II}/2} \left[ \sum_{p \in \Omega_g} n_p (2H_{pp} + J_{pp}) + \sum_{q,p \in \Omega_g q \ne p} \underbrace{\Pi_{qp}^{g}}_{C^K_{pq}} K_{pq} \right] + \sum_{f \neq g}^{N_{II}/2} \sum_{p \in \Omega_f} \sum_{q \in \Omega_g} (\underbrace{2n_q n_p}_{C^J_{q p}} J_{pq}- \underbrace{n_q n_p}_{C^K_{q p}} K_{pq})
\end{equation}


\begin{equation}
C^J_{ij} = \left\{
  \begin{array}{lll}
  2 n_i n_j & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
  0         & \mathrm{si\ } i,j \in \Omega_{g}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
C^K_{ij} = \left\{
  \begin{array}{lll}
  n_i n_j   & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
  -\Pi_{ij}       & \mathrm{si\ } i,j \in \Omega_{g}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
\Pi_{ij} = \left\{
  \begin{array}{lll}
   \sqrt{n_i n_j}      & \mathrm{si\ } i,j \in \Omega_{g} i=j\ \text{ó}\ i\ \text{y}\ j > \frac{N_{II}}{2}\\
  -\sqrt{n_i n_j}      & \mathrm{si\ } i,j \in \Omega_{g} i\ \text{ó}\ j \leq \frac{N_{II}}{2}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
\frac{dC^J_{ij}}{d\gamma_k} = \left\{
  \begin{array}{lll}
  2 \frac{d n_i}{d \gamma_k} n_j & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
  0         & \mathrm{si\ } i,j \in \Omega_{g}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
\frac{d C^K_{ij}}{d \gamma_k} = \left\{
  \begin{array}{lll}
  \frac{dn_i}{d\gamma_k} n_j & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
 -\frac{d \sqrt{n_i n_j}}{d \gamma_k} = -\frac{1}{2\sqrt{n_i}} \frac{d n_i}{d \gamma_k} \sqrt{n_j} & \mathrm{si\ } i,j \in \Omega_{g} i=j\ \text{ó}\ i\ \text{y}\ j > \frac{N_{II}}{2}\\
  \frac{d \sqrt{n_i n_j}}{d \gamma_k} =  \frac{1}{2\sqrt{n_i}} \frac{d n_i}{d \gamma_k} \sqrt{n_j} & \mathrm{si\ } i,j \in \Omega_{g} i\ \text{ó}\ j \leq \frac{N_{II}}{2}\\  \end{array}
  \right.
\end{equation}

In [31]:
#CJCKD5
def CJCKD5(n):    
    
    cj12 = 2*np.einsum('i,j->ij',n,n)
    ck12 = np.einsum('i,j->ij',n,n)    
    
    for l in range(ndoc):            
        ldx = no1 + l
        # inicio y fin de los orbitales acoplados a los fuertemente ocupados
        ll = no1 + ndns+ncwo*(ndoc-l-1)
        ul = no1 + ndns+ncwo*(ndoc-l)

        cj12[ldx,ll:ul] = 0    
        cj12[ll:ul,ldx] = 0    
    
        cj12[ll:ul,ll:ul] = 0    
        
        ck12[ldx,ll:ul] = np.sqrt(n[ldx]*n[ll:ul])
        ck12[ll:ul,ldx] = np.sqrt(n[ldx]*n[ll:ul])

        ck12[ll:ul,ll:ul] = -np.outer(np.sqrt(n[ll:ul]),np.sqrt(n[ll:ul]))

    return cj12,ck12        
        
def der_CJCKD5(n,gamma,dn_dgamma):

    Dcj12r = 2*np.einsum('ik,j->ijk',dn_dgamma,n)    
    Dck12r = np.einsum('ik,j->ijk',dn_dgamma,n)    

    for l in range(ndoc):            
        ldx = no1 + l

        # inicio y fin de los orbitales acoplados a los fuertemente ocupados
        ll = no1 + ndns+ncwo*(ndoc-l-1)
        ul = no1 + ndns+ncwo*(ndoc-l)

        Dcj12r[ldx,ll:ul,:nv] = 0
        Dcj12r[ll:ul,ldx,:nv] = 0

        Dcj12r[ll:ul,ll:ul,:nv] = 0   
        
        a = n[ldx] 
        if(a<10**-12):
            a = 10**-12
        b = n[ll:ul]
        if(b<10**-12):
            b = 10**-12
        
        Dck12r[ldx,ll:ul,:nv] = 1/2 * np.sqrt(1/a)*dn_dgamma[ldx,:nv]*np.sqrt(n[ll:ul])
#        Dck12r[ll:ul,ldx,:nv] = 1/2 * np.sqrt(1/b)*dn_dgamma[ll:ul,:nv]*np.sqrt(n[ldx])
        Dck12r[ll:ul,ldx,:nv] = 1/2 * np.sqrt(1/b)*dn_dgamma[ll:ul,:nv]*np.sqrt(n[ldx])
        
        for k in range(nv):
            Dck12r[ll:ul,ll:ul,k] = -1/2 * dn_dgamma[ll:ul,k]   
                        
    return Dcj12r,Dck12r

Para PNOF7

\begin{equation}
E^{PNOF7} = \sum_{g=1}^{N_{II}/2} \left[ \sum_{p \in \Omega_g} n_p (2H_{pp} + J_{pp}) + \sum_{q,p \in \Omega_g q \ne p} \underbrace{\Pi_{qp}^{g}}_{C^K_{pq}} K_{pq} \right] + \sum_{f \neq g}^{N_{II}/2} \sum_{p \in \Omega_f} \sum_{q \in \Omega_g} (\underbrace{2n_q n_p}_{C^J_{q p}} J_{pq}- \underbrace{n_q n_p + \Phi_q \Phi_p}_{C^K_{q p}} K_{pq})
\end{equation}

\begin{equation}
\Phi_i = \sqrt{n_i(1-n_i)}
\end{equation}


\begin{equation}
C^J_{ij} = \left\{
  \begin{array}{lll}
  2 n_i n_j & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
  0         & \mathrm{si\ } i,j \in \Omega_{g}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
C^K_{ij} = \left\{
  \begin{array}{lll}
  n_i n_j + \Phi_i \Phi_j   & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
  -\Pi_{ij}       & \mathrm{si\ } i,j \in \Omega_{g}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
\Pi_{ij} = \left\{
  \begin{array}{lll}
   \sqrt{n_i n_j}      & \mathrm{si\ } i,j \in \Omega_{g} i=j\ \text{ó}\ i\ \text{y}\ j > \frac{N_{II}}{2}\\
  -\sqrt{n_i n_j}      & \mathrm{si\ } i,j \in \Omega_{g} i\ \text{ó}\ j \leq \frac{N_{II}}{2}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
\frac{d \Phi_i}{d\gamma_k} = \left\{
  \begin{array}{lll}
  \frac{d \sqrt{n_i(1-n_i)}}{d \gamma_k} = \frac{1}{2 \sqrt{n_i(1-n_i)}} (1 - 2n_i) \frac{d n_i}{d \gamma_i} \delta_{ik} & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
  0         & \mathrm{si\ } i,j \in \Omega_{g}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
\frac{dC^J_{ij}}{d\gamma_k} = \left\{
  \begin{array}{lll}
  2 \frac{d n_i}{d \gamma_k} n_j & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
  0         & \mathrm{si\ } i,j \in \Omega_{g}\\
  \end{array}
  \right.
\end{equation}

\begin{equation}
\frac{d C^K_{ij}}{d \gamma_k} = \left\{
  \begin{array}{lll}
  \frac{dn_i}{d\gamma_k} n_j + \frac{d\Phi_i}{d\gamma_k} \Phi_j & \mathrm{si\ } i \in \Omega_{g} j \in \Omega_{f} f \ne g\\
 -\frac{d \sqrt{n_i n_j}}{d \gamma_k} = -\frac{1}{2\sqrt{n_i}} \frac{d n_i}{d \gamma_k} \sqrt{n_j} & \mathrm{si\ } i,j \in \Omega_{g} i=j\ \text{ó}\ i\ \text{y}\ j > \frac{N_{II}}{2}\\
  \frac{d \sqrt{n_i n_j}}{d \gamma_k} =  \frac{1}{2\sqrt{n_i}} \frac{d n_i}{d \gamma_k} \sqrt{n_j} & \mathrm{si\ } i,j \in \Omega_{g} i\ \text{ó}\ j \leq \frac{N_{II}}{2}\\  \end{array}
  \right.
\end{equation}

In [32]:
#CJCKD7
def CJCKD7(n):    
    
    fi = n*(1-n)
    fi[fi<=0] = 0
    fi = np.sqrt(fi)      
    
    cj12 = 2*np.einsum('i,j->ij',n,n)
    ck12 = np.einsum('i,j->ij',n,n)    
    ck12 += np.einsum('i,j->ij',fi,fi)
    
    for l in range(ndoc):            
        ldx = no1 + l
        # inicio y fin de los orbitales acoplados a los fuertemente ocupados
        ll = no1 + ndns+ncwo*(ndoc-l-1)
        ul = no1 + ndns+ncwo*(ndoc-l)

        cj12[ldx,ll:ul] = 0    
        cj12[ll:ul,ldx] = 0    
    
        cj12[ll:ul,ll:ul] = 0    
        
        ck12[ldx,ll:ul] = np.sqrt(n[ldx]*n[ll:ul])
        ck12[ll:ul,ldx] = np.sqrt(n[ldx]*n[ll:ul])

        ck12[ll:ul,ll:ul] = -np.outer(np.sqrt(n[ll:ul]),np.sqrt(n[ll:ul]))

    return cj12,ck12        
        
def der_CJCKD7(n,dn_dgamma):
    
    fi = n*(1-n)
    fi[fi<=0] = 0
    fi = np.sqrt(fi)      
            
    dfi_dgamma = np.zeros((nbf5,nv))
    for i in range(no1,nbf5):
        a = fi[i]
        if(a < 10**-12):
            a = 10**-12
        for k in range(nv):
            dfi_dgamma[i,k] = 1/(2*a)*(1-2*n[i])*dn_dgamma[i][k]
    
    Dcj12r = 2*np.einsum('ik,j->ijk',dn_dgamma,n)    
    Dck12r = np.einsum('ik,j->ijk',dn_dgamma,n)    
    Dck12r += np.einsum('ik,j->ijk',dfi_dgamma,fi)    

    for l in range(ndoc):            
        ldx = no1 + l

        # inicio y fin de los orbitales acoplados a los fuertemente ocupados
        ll = no1 + ndns+ncwo*(ndoc-l-1)
        ul = no1 + ndns+ncwo*(ndoc-l)

        Dcj12r[ldx,ll:ul,:nv] = 0
        Dcj12r[ll:ul,ldx,:nv] = 0

        Dcj12r[ll:ul,ll:ul,:nv] = 0   

        a = n[ldx] 
        if(a<10**-12):
            a = 10**-12
        b = n[ll:ul]
        if(b<10**-12):
            b = 10**-12        
        
        Dck12r[ldx,ll:ul,:nv] = 1/2 * np.sqrt(1/a)*dn_dgamma[ldx,:nv]*np.sqrt(n[ll:ul])
        Dck12r[ll:ul,ldx,:nv] = 1/2 * np.sqrt(1/b)*dn_dgamma[ll:ul,:nv]*np.sqrt(n[ldx])
        
        for k in range(nv):
            Dck12r[ll:ul,ll:ul,k] = - 1/2 * dn_dgamma[ll:ul,k]
                        
    return Dcj12r,Dck12r

Creamos un seleccionador de PNOF

In [33]:
def PNOFi_selector(PNOFi,n):
    if(PNOFi==5):
        cj12,ck12 = CJCKD5(n)
    if(PNOFi==7):
        cj12,ck12 = CJCKD7(n)
        
    return cj12,ck12

def der_PNOFi_selector(PNOFi,n,dn_dgamma):
    if(PNOFi==5):
        Dcj12r,Dck12r = der_CJCKD5(n,dn_dgamma)
    if(PNOFi==7):
        Dcj12r,Dck12r = der_CJCKD7(n,dn_dgamma)
        
    return Dcj12r,Dck12r

## Optimización de Ocupaciones

Declaramos el número de variables en la optimización de ocupaciones
\begin{equation}
N_v = N_{cwo}N_{doc}
\end{equation}

In [34]:
nv = ncwo*ndoc

Definimos una función para calcular $J_{MO}$ y $K_{MO}$
\begin{eqnarray}
{J_{MO}}_{ij} &=& \sum_{\mu\nu} D^{(j)}_{\mu\nu} J^{(i)}_{\mu\nu}\\
{K_{MO}}_{ij} &=& \sum_{\mu\sigma} D^{(j)}_{\mu\sigma} K^{(i)}_{\mu\sigma}\\
{H_{core}}_{i} &=& \sum_{\mu\nu} D^{(i)}_{\mu\nu} H_{\mu\nu}
\end{eqnarray}


In [35]:
def computeJKH_core_MO(C,H):

    #denmatj    
    D = np.einsum('mi,ni->imn', C[:,0:nbf5], C[:,0:nbf5], optimize=True)

    #QJMATm
    J = np.einsum('isl,mnsl->imn', D, I, optimize=True)    
    J_MO = np.einsum('jmn,imn->ij', D, J, optimize=True)    
        
    #QKMATm        
    K = np.einsum('inl,mnsl->ims', D, I, optimize=True)    
    K_MO = np.einsum('jms,ims->ij', D, K, optimize=True)        

    #QHMATm
    H_core = np.einsum('imn,mn->i', D, H, optimize=True)

    return J_MO,K_MO,H_core

Definimos una función que calcule los números de ocupación y sus derivadas respecto a $\gamma$

RO:
\begin{equation}
n_i = \left\{
  \begin{array}{lll}
   1      & \mathrm{si\ } i \in [1,N_{o1}]\\
   \frac{1}{2} (1 + cos^2(\gamma_{i-N_{doc}}))     & \mathrm{si\ } i \in (N_{o1},N_{\beta}]\\
   \frac{1}{2} sin^2(\gamma_{N_{doc}-i-1})     & \mathrm{si\ } i \in (N_{\alpha},N_{bf5}]\\
  \end{array}
  \right.
\end{equation}


DRO:
\begin{equation}
\frac{d n_i}{d \gamma_k} = \left\{
  \begin{array}{lll}
   0      & \mathrm{si\ } i \in [1,N_{o1}]\\
   -\frac{1}{2} sin(2\gamma_{i-N_{doc}}) \delta_{ik}     & \mathrm{si\ } i \in (N_{o1},N_{\beta}]\\
   \frac{1}{2} sin(2\gamma_{N_{doc}-i-1})     & \mathrm{si\ } i \in (N_{\alpha},N_{bf5}]\\
  \end{array}
  \right.
\end{equation}

In [36]:
def ocupacion(gamma):
        
    n = np.zeros((nbf5))
    dni_dgammai = np.zeros((nbf5))

    n[0:no1] = 1                                              # [1,no1]

    n[no1:no1+ndoc] = 1/2 * (1 + np.cos(gamma[:ndoc])**2)     # (no1,no1+ndoc]
    dni_dgammai[no1:no1+ndoc] = - 1/2 * np.sin(2*gamma[:ndoc])
       
    if(ncwo==1):
        dn_dgamma = np.zeros((nbf5,nv))    

        for i in range(ndoc):
            dn_dgamma[no1+i][i] = dni_dgammai[no1+i]
            #cwo
            icf = nalpha+ndoc - i - 1
            n[icf] = 1/2*np.sin(gamma[i])**2
            dni_dgammai[icf]  = 1/2*np.sin(2*gamma[i])
            dn_dgamma[icf][i] = dni_dgammai[icf]

    return n,dn_dgamma

**Generamos gamma**

\begin{equation}
\gamma_{k} = \left\{
  \begin{array}{lll}
  cos^{-1}(\sqrt{2\times0.999-1})      & \mathrm{si\ } k \in N_{doc}  \\
  sin^{-1}(\sqrt{\frac{1}{N_{cwo}-j+1}})     & \mathrm{si\ } k = N_{doc} + (i-1)(N_{cwo}-1)+j;i\in N_{doc};j\in N_{cwo}-1
  \end{array}
  \right.
\end{equation}

Esto se hace considerando la relación
\begin{equation}
n_g = \frac{1}{2} (1 + cos^2 \gamma_g) ; g= 1,2,\cdots,N_{II}/2
\end{equation}


In [37]:
gamma = np.zeros((nbf5))
for i in range(ndoc):
    gamma[i] = np.arccos(np.sqrt(2.0*0.999-1.0))
    for j in range(ncwo-1):
        ig = ndoc+(i)*(ncwo-1)+j
        gamma[ig] = np.arcsin(np.sqrt(1.0/(ncwo-j)))        

**Definimos una función que recibe ($\gamma$) y calcula la energía**

\begin{eqnarray}
E &=& \sum^{N_{o1}}_{i=1} \left[ n_i (2{H_{core}}_i + {J_{MO}}_{ii}) + \sum^{N_{bf5}}_{j \neq i} C^J_{ij} {J_{MO}}_{ji} - C^K_{ij} {K_{MO}}_{ji} \right]\\
&+& \sum^{N_{doc}}_{i=N_{o1}+1} \left[ n_i (2{H_{core}}_i + {J_{MO}}_{ii}) + \sum^{N_{bf5}}_{j \neq i} C^J_{ij} {J_{MO}}_{ji} - C^K_{ij} {K_{MO}}_{ji} \right]\\
&+& \sum^{(N_{doc}+1)N_{cwo}}_{i=N_{doc}+1} \left[ n_i (2{H_{core}}_i + {J_{MO}}_{ii}) + \sum^{N_{bf5}}_{j \neq i} C^J_{ij} {J_{MO}}_{ji} - C^K_{ij} {K_{MO}}_{ji} \right]\\
\end{eqnarray}

In [38]:
#calce
def calce(gamma,J_MO,K_MO,H_core,PNOFi):
    
    n,dn_dgamma = ocupacion(gamma)
    cj12,ck12 = PNOFi_selector(PNOFi,n)
    
    E = 0

    # 2H + J
    E = E + np.einsum('i,i',n[:nbeta],2*H_core[:nbeta]+np.diagonal(J_MO)[:nbeta]) # [0,Nbeta]
    E = E + np.einsum('i,i',n[nbeta:nalpha],2*H_core[nbeta:nalpha])               # (Nbeta,Nalpha]
    E = E + np.einsum('i,i',n[nalpha:nbf5],2*H_core[nalpha:nbf5]+np.diagonal(J_MO)[nalpha:nbf5]) # (Nalpha,Nbf5)

    #C^J JMO
    E = E + np.einsum('ij,ji',cj12,J_MO) # sum_ij
    E = E - np.einsum('ii,ii',cj12,J_MO) # Quita i=j

    #C^K KMO     
    E = E - np.einsum('ij,ji',ck12,K_MO) # sum_ij
    E = E + np.einsum('ii,ii',ck12,K_MO) # Quita i=j
            
    return E

Definimos una función que recibe ($\gamma$) y calcula el gradiente

\begin{equation}
\frac{dE}{d\gamma_{k}} = \sum_{i=N_{o1}+1}^{N_{\beta}} \left[ \frac{dn_i}{d \gamma_{k}} (2{H_{core}}_{i} + {J_{MO}}_{ii}) + \sum_{j \neq i}^{N_{bf5}} 2 \frac{d C^J_{ij}}{d \gamma_{k}}{J_{MO}}_{ji} - 2 \frac{d C^K_{ij}}{d \gamma_{k}}{K_{MO}}_{ji} \right] + \sum_{i=N_{\alpha}+1}^{N_{bf5}} \left[ \frac{dn_i}{d \gamma_{k}} (2{H_{core}}_{i} + {J_{MO}}_{ii}) + \sum_{j \neq i}^{N_{bf5}} 2 \frac{d C^J_{ij}}{d \gamma_{k}}{J_{MO}}_{ji} - 2 \frac{d C^K_{ij}}{d \gamma_{k}}{K_{MO}}_{ji} \right]
\end{equation}

In [39]:
#calcg
def calcg(gamma,J_MO,K_MO,H_core,PNOFi):
    
    grad = np.zeros((nv))

    n,dn_dgamma = ocupacion(gamma)    
    Dcj12r,Dck12r = der_PNOFi_selector(PNOFi,n,dn_dgamma)

    # dn_dgamma (2H+J)
    grad += np.einsum('ik,i->k',dn_dgamma[no1:nbeta,:nv],2*H_core[no1:nbeta]+np.diagonal(J_MO)[no1:nbeta],optimize=True) # [0,Nbeta]
    grad += np.einsum('ik,i->k',dn_dgamma[nalpha:nbf5,:nv],2*H_core[nalpha:nbf5]+np.diagonal(J_MO)[nalpha:nbf5],optimize=True) # [Nalpha,Nbf5]

    # 2 dCJ_dgamma J_MO
    grad += 2*np.einsum('ijk,ji->k',Dcj12r[no1:nbeta,:nbf5,:nv],J_MO[:nbf5,no1:nbeta],optimize=True)
    grad -= 2*np.einsum('iik,ii->k',Dcj12r[no1:nbeta,no1:nbeta,:nv],J_MO[no1:nbeta,no1:nbeta],optimize=True)

    grad += 2*np.einsum('ijk,ji->k',Dcj12r[nalpha:nbf5,:nbf5,:nv],J_MO[:nbf5,nalpha:nbf5],optimize=True)
    grad -= 2*np.einsum('iik,ii->k',Dcj12r[nalpha:nbf5,nalpha:nbf5,:nv],J_MO[nalpha:nbf5,nalpha:nbf5],optimize=True)
      
    # -2 dCK_dgamma K_MO    
    grad -= 2*np.einsum('ijk,ji->k',Dck12r[no1:nbeta,:nbf5,:nv],K_MO[:nbf5,no1:nbeta],optimize=True)
    grad += 2*np.einsum('iik,ii->k',Dck12r[no1:nbeta,no1:nbeta,:nv],K_MO[no1:nbeta,no1:nbeta],optimize=True)

    grad -= 2*np.einsum('ijk,ji->k',Dck12r[nalpha:nbf5,:nbf5,:nv],K_MO[:nbf5,nalpha:nbf5],optimize=True)
    grad += 2*np.einsum('iik,ii->k',Dck12r[nalpha:nbf5,nalpha:nbf5,:nv],K_MO[nalpha:nbf5,nalpha:nbf5],optimize=True)   

    return grad

**Definimos una función que optimiza los numeros de ocupación utilizando la energía como función objetivo.**

In [40]:
def occoptr(gamma,firstcall,convgdelag,elag,C,H):
        
    J_MO,K_MO,H_core = computeJKH_core_MO(C,H)
    
    if (not convgdelag):
        if(gradient=="analytical"):
            res = minimize(calce, gamma[:nv], args=(J_MO,K_MO,H_core,PNOFi), jac=calcg, method='CG')
        elif(gradient=="numerical"):
            res = minimize(calce, gamma[:nv], args=(J_MO,K_MO,H_core,PNOFi),  method='CG')
        gamma = res.x
    n,DR = ocupacion(gamma)
    cj12,ck12 = PNOFi_selector(PNOFi,n)
        
    if (firstcall):
        elag_diag = np.zeros((nbf))
               
        # RO (H_core + J)
        elag_diag[:nbeta] = np.einsum('i,i->i',n[:nbeta],H_core[:nbeta]+np.diagonal(J_MO)[:nbeta])        
        elag_diag[nbeta:nalpha] = np.einsum('i,i->i',n[nbeta:nalpha],H_core[nbeta:nalpha])        
        elag_diag[nalpha:nbf5] = np.einsum('i,i->i',n[nalpha:nbf5],H_core[nalpha:nbf5]+np.diagonal(J_MO)[nalpha:nbf5])        

        # CJ12 J_MO
        elag_diag[:nbf5] += np.einsum('ij,ji->i',cj12,J_MO)
        elag_diag[:nbf5] -= np.einsum('ii,ii->i',cj12,J_MO)

        # CK12 K_MO
        elag_diag[:nbf5] -= np.einsum('ij,ji->i',ck12,K_MO)
        elag_diag[:nbf5] += np.einsum('ii,ii->i',ck12,K_MO)
        
#        for i in range(nbf):
#            elag[i][i] = elag_diag[i]
    
    return gamma,elag,n,cj12,ck12

Hacemos una primera optimización de los números de ocupación

In [41]:
gamma,elag,n,cj12,ck12 = occoptr(gamma,True,False,elag,C,H)

## Optimización Orbital y (de ocupaciones)

Definimos una función para DIIS

In [42]:
def fmiug_diis(fk,fmiug,idiis,bdiis,maxdiff):

    fk[idiis,0:noptorb,0:noptorb] = fmiug[0:noptorb,0:noptorb]
    for m in range(idiis+1):
        bdiis[m][idiis] = 0
        for i in range(noptorb):
            for j in range(i):
                bdiis[m][idiis] = bdiis[m][idiis] + fk[m][i][j]*fk[idiis][j][i]
        bdiis[idiis][m] = bdiis[m][idiis]
        bdiis[m][idiis+1] = -1
        bdiis[idiis+1][m] = -1
    bdiis[idiis+1][idiis+1] = 0

    if(idiis>=ndiis):
        cdiis = np.zeros((idiis+2))
        cdiis[0:idiis+1] = 0
        cdiis[idiis+1] = -1
        x = np.linalg.solve(bdiis[0:idiis+2,0:idiis+2],cdiis[0:idiis+2])

        for i in range(noptorb):
            for j in range(i):
                fmiug[i][j] = 0
                for k in range(idiis+1):
                    fmiug[i][j] = fmiug[i][j] + x[k]*fk[k][i][j]
                fmiug[j][i] = fmiug[i][j]

    if(idiis>=ndiis):
        if(perdiis):
            idiis = 0
    else:
        idiis = idiis + 1
    
    return fk,fmiug,idiis,bdiis

Creamos una función para la optimización orbital

In [43]:
def orboptr(C,n,cj12,ck12,E_old,E_diff,sumdiff_old,i_ext,itlim,nzeros,fmiug0):      
    
    convgdelag = False

    E,elag,sumdiff,maxdiff = ENERGY1r(C,n,cj12,ck12)

    #E_diff = E-E_old                    
    #P_CONV = abs(E_diff)
    #E_old = E
    
    if(maxdiff<threshl and abs(E_diff)<threshe):
        convgdelag = True
        print('{:6d} {:6d} {:14.8f} {:14.8f} {:14.8f} {:14.8f}'.format(i_ext,0,E,E+E_nuc,E_diff,maxdiff))        
        return convgdelag,E_old,E_diff,sumdiff_old,itlim,nzeros,fmiug0,C    
                
    if (scaling and i_ext>1 and i_ext >= itlim and sumdiff > sumdiff_old):
        nzeros = nzeros + 1
        itlim = (i_ext) + itziter
        if (nzeros>nzerosm):
            nzeros = nzerosr
    sumdiff_old = sumdiff
    
    if i_ext==0:
        maxlp = 1
    else:
        maxlp = maxloop
    
    fmiug = np.zeros((noptorb,noptorb))
    fk = np.zeros((30,noptorb,noptorb))
    bdiis = np.zeros((31,31))
    cdiis = np.zeros((31))        
    iloop = 0
    idiis = 0
    
    for i_int in range(maxlp):
        iloop = iloop + 1
        E_old2 = E

        #scaling
        if(scaling):
            fmiug = fmiug_scaling(fmiug0,elag,i_ext,nzeros)
        if(diis and maxdiff < thdiis):
            fk,fmiug,idiis,bdiis = fmiug_diis(fk,fmiug,idiis,bdiis,maxdiff)   
          
        eigval, eigvec = np.linalg.eigh(fmiug)
        fmiug0 = eigval
                
        C = np.matmul(C,eigvec)

        E,elag,sumdiff,maxdiff = ENERGY1r(C,n,cj12,ck12)

        E_diff2 = E-E_old2                    
                       
        if(abs(E_diff2)<threshec or i_int==maxlp-1):
            E_diff = E-E_old
            E_old = E
            print('{:6d} {:6d} {:14.8f} {:14.8f} {:14.8f} {:14.8f}'.format(i_ext+1,i_int,E,E+E_nuc,E_diff,maxdiff))
            break    
    
    return convgdelag,E_old,E_diff,sumdiff_old,itlim,nzeros,fmiug0,C

## SCF de PNOF

In [44]:
iloop = 0
itlim = 0
E_old = E
E_diff = 99999
sumdiff_old = 0

print('{:^7} {:^7} {:^14} {:^14} {:^14} {:^14}'.format("Nitext","Nitint","Eelec","Etot","Ediff","maxdiff"))

for i_ext in range(maxit):
        
    #orboptr
    convgdelag,E_old,E_diff,sumdiff_old,itlim,nzeros,fmiug0,C = orboptr(C,n,cj12,ck12,E_old,E_diff,sumdiff_old,i_ext,itlim,nzeros,fmiug0)     
    
    #occopt
    gamma,elag,n,cj12,ck12 = occoptr(gamma,False,convgdelag,elag,C,H)
        
    if(convgdelag):
        break

Nitext  Nitint      Eelec           Etot          Ediff         maxdiff    
     1      0   -85.38391403   -76.02930333    -0.00201648     0.00746295
     2     29   -85.28431177   -75.92970107     0.09960226     0.21818665
     3     29   -85.41279790   -76.05818719    -0.12848613     0.02390747
     4     29   -85.43441096   -76.07980025    -0.02161306     0.01351943
     5     29   -85.43898819   -76.08437749    -0.00457724     0.02519197
     6     29   -85.44599023   -76.09137952    -0.00700204     0.01432708
     7     29   -85.44942635   -76.09481565    -0.00343612     0.01642054
     8     29   -85.44972318   -76.09511248    -0.00029683     0.02306486
     9     29   -85.45036955   -76.09575885    -0.00064637     0.01754294
    10     29   -85.45107700   -76.09646630    -0.00070745     0.01618773
    11     29   -85.45138400   -76.09677329    -0.00030699     0.01526692
    12     29   -85.45096030   -76.09634960     0.00042370     0.01197825
    13     29   -85.45238884   -76.0

In [45]:
for i in range(C.shape[0]):
    print("------------------")
    for j in range(C.shape[0]):
        print(i,j,C[j][i])

------------------
0 0 1.0010264944231435
0 1 -0.006351551238227851
0 2 -0.009982188924751523
0 3 -9.165356260556314e-05
0 4 -1.984283449066226e-06
0 5 -0.005047188336178387
0 6 -7.215850926367547e-05
0 7 -1.091763918459567e-06
0 8 -0.003118459960060483
0 9 -0.0013355054339851357
0 10 4.12711363164297e-07
0 11 3.783553734557912e-06
0 12 -0.0011198483608865714
0 13 7.451273264676405e-07
0 14 -0.0009937800142861095
0 15 -0.0017588727114651087
0 16 0.00033518951949393773
0 17 -5.309338083745645e-06
0 18 0.0008296432268480996
0 19 -0.0007675228113553847
0 20 -0.0017570670730814963
0 21 0.00033588870680978524
0 22 -4.815734898468971e-06
0 23 -0.0008303161008649496
0 24 -0.0007670929835593733
------------------
1 0 -0.002681861466461302
1 1 -0.14453410539838057
1 2 0.011329083202887766
1 3 0.00016076041008747298
1 4 -0.3513492019244533
1 5 0.31864224432565336
1 6 0.0001224755315266911
1 7 -0.14459822866956876
1 8 0.17095524627628042
1 9 0.0040042354870226575
1 10 -7.864953999364332e-06
1 11 

21 21 0.2721504027231094
21 22 0.10490480530202693
21 23 0.5653704377442234
21 24 0.4184361448384353
------------------
22 0 -0.011573954421126218
22 1 -0.18696375438434973
22 2 -0.2865754088599853
22 3 -0.02320989046774642
22 4 -0.15140462136949598
22 5 -0.012350924246243586
22 6 -0.052702121470825536
22 7 -0.5308869425642514
22 8 -0.0999239509091582
22 9 0.08468391097956283
22 10 0.7062863095506559
22 11 0.3417689606018126
22 12 1.129519609075294
22 13 0.6459393188621503
22 14 -0.33475511951429526
22 15 0.027397726094829617
22 16 0.4162425135132153
22 17 -0.20017286269410287
22 18 -0.18129413472749195
22 19 0.11065368171609839
22 20 -0.647288383114429
22 21 -0.09349968779375326
22 22 0.3938934608392205
22 23 -0.13034628617582653
22 24 -0.4929762512062725
------------------
23 0 -0.23174192789415715
23 1 -1.158923080475359
23 2 -0.17910429459513097
23 3 -0.02584901038471834
23 4 0.18738349226343293
23 5 -0.1684403393947974
23 6 -0.01937305041539043
23 7 0.39426334323802603
23 8 -0.597

# Proyección

In [46]:
# Set computation options
old_basis = wfn.basisset()

psi4.set_options({'basis': 'cc-pVDZ',
                  'scf_type': 'pk',
                  'e_convergence': 1e-8})

In [75]:
from scipy.linalg import orth
from numpy.linalg import matrix_power
import scipy

#aux = psi4.core.BasisSet.build(mol, "ORBITAL", "")
#old_basis = wfn.basisset()

wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis'))
mints = psi4.core.MintsHelper(wfn.basisset())

M = np.asarray(mints.ao_overlap(wfn.basisset(),old_basis))

Sinv = mints.ao_overlap()
Sm12 = mints.ao_overlap()
Sp12 = mints.ao_overlap()
Sinv.power(-1.0, 1.e-14)
Sm12.power(-0.5, 1.e-14)
Sp12.power(0.5, 1.e-14)

Cnew = np.matmul(np.matmul(Sinv,M),C)

#Cnew = np.matmul(Sp12,Cnew)

#Cnew = orth(Cnew)

#Cnew = np.matmul(Sm12,Cnew)

#Q = np.matmul(Cnew.T,Cnew)
#Q = psi4.core.Matrix.from_array(Q)
#Q.power(-0.5,1.e-16)

#Cneww = scipy.linalg.orth(Cneww)
C = np.zeros((Cnew.shape[0],Cnew.shape[0]))
C[:,:Cnew.shape[1]] = Cnew

for i in range(Cnew.shape[1]):
    print("------------------")
    for j in range(Cnew.shape[0]):
        print(i,j,Cnew[j][i])#,Cneww[j][i])        

------------------
0 0 -1.0007526590398925
0 1 0.007004106285952112
0 2 0.014946642394906232
0 3 0.0003390923141355264
0 4 5.902801448168143e-06
0 5 0.004205042446625395
0 6 0.0002708883619919235
0 7 2.577794046205596e-06
0 8 0.0010709696313506477
0 9 -0.0018626123963160594
0 10 -3.1952577842846245e-17
0 11 -1.0810613452765647e-07
0 12 -0.0019327541568239463
0 13 5.521541501232104e-08
0 14 -0.0019766443235727377
0 15 0.000760342787310239
0 16 -0.0016064609259680454
0 17 -1.567309776928905e-07
0 18 -0.00015321997888819138
0 19 9.79208477951968e-06
0 20 0.000751397084400426
0 21 -0.001608990754080743
0 22 -1.5673097782057608e-07
0 23 0.0001529909952379998
0 24 9.559936351009608e-06
------------------
1 0 0.0009944693088530408
1 1 0.13239308079720757
1 2 0.013988256389170345
1 3 -0.00035994975543089333
1 4 0.3545192315209392
1 5 -0.3344415352703767
1 6 -0.00018344993892451745
1 7 0.15724632444871184
1 8 -0.20506145753343943
1 9 0.005983674837392438
1 10 1.0985125443005216e-17
1 11 3.05593

Construimos la función de onda y **evaluamos matrices e integrales en orbital atómico**, $S$, $T$, $V$, $H$, $(\mu\nu|\sigma\lambda)$

In [76]:
# Wavefunction
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis'))

# Integrador
mints = psi4.core.MintsHelper(wfn.basisset())

# Overlap, Kinetics, Potential
S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
H = T + V

# Integrales de Repulsión Electrónica, ERIs (mu nu | sigma lambda)
I = np.asarray(mints.ao_eri())

# Energía Nuclear
E_nuc = mol.nuclear_repulsion_energy()

# Variables

In [77]:
natoms = mol.natom()
nbf = S.shape[0]
nalpha = wfn.nalpha()
nbeta = wfn.nbeta()
ne = nalpha + nbeta
mul = mol.multiplicity()
no1 = 0 #Number of inactive doubly occupied orbitals | Se puede variar
for i in range(natoms):
    Z = mol.Z(i)
    if ( 1<=Z and Z<=  2):
        no1 += 0           # H-He
    elif ( 3<=Z and Z<= 10):
        no1 +=  1          # Li-Ne
    elif (11<=Z and Z<= 18):
        no1 +=  5          # Na-Ar
    elif(19<=Z and Z<= 36):
        no1 +=  9          # K-Kr
    elif(37<=Z and Z<= 49):
        no1 += 18          # Rb-In
    elif(50<=Z and Z<= 54):
        no1 += 23          # Sn-Xe
    elif(55<=Z and Z<= 71):
        no1 += 27          # Cs-Lu
    elif(72<=Z and Z<= 81):
        no1 += 30          # Hf-Tl
    elif(82<=Z and Z<= 86):
        no1 += 39          # Pb-Rn
    elif(87<=Z and Z<=109):
        no1 += 43          # Fr-Mt
    
ndoc = nbeta   -   no1
nsoc = nalpha  -   nbeta
ndns = ndoc    +   nsoc
nvir = nbf     -   nalpha

ncwo = 1
if(ne==2):
    ncwo= -1
if(ndns!=0):
    if(ndoc>0):
        if(ncwo!=1):
            if(ncwo==-1 or ncwo > nvir/ndoc):
                ncwo = int(nvir/ndoc)
    else:
        ncwo = 0

nac = ndoc * (1 + ncwo)
nbf5 = no1 + nac + nsoc   #JFHLY warning: nbf must be >nbf5
no0 = nbf - nbf5

noptorb = nbf     

closed = (nbeta == (ne+mul-1)/2 and nalpha == (ne-mul+1)/2)

Establecemos algunos parámetros

In [78]:
maxit = 1000  # Número máximo de iteraciones de Occ-SCF
no1 = no1     # Número de orbitales inactivos con ocupación 1
thresheid = 10**-8 # Convergencia de la energía total
maxitid = 30  # Número máximo de iteraciones externas
ipnof = 5     # PNOFi a calcular
threshl = 10**-4   # Convergencia de los multiplicadores de Lagrange
threshe = 10**-6   # Convergencia de la energía
threshec = 10**-10 # Convergencia  de la energía en optimización orbital
threshen = 10**-10 # Convergencia  de la energía en optimización de ocupaciones
scaling = True     # Scaling for f
nzeros = 1
nzerosm = 5
nzerosr = 2
itziter = 10        # Iteraciones para scaling constante
diis = True         # DIIS en optimización orbital
thdiis = 10**-3     # Para iniciar DIIS
ndiis = 5           # Número de ciclos para interpolar matriz de Fock generalizada en DIIS
perdiis = True      # Aplica DIIS cada NDIIS (True) o después de NDIIS (False) 
ncwo = ncwo         # Número de orbitales débilmente ocupados acoplados a cada orbital fueremtente ocupado 
noptorb = noptorb   # Número de orbitales a optimizar Nbf5 <= Noptorb <= Nbf
scaling = True

Obtenemos un Guess acorde a la ecuación

\begin{equation}
HC = SC\varepsilon
\end{equation}

In [79]:
import numpy as np
from numpy.linalg import lstsq
from scipy.linalg import orth

E_i,Ctmp = eigh(H, S)
C[:,Cnew.shape[1]:] = Ctmp[:,Cnew.shape[1]:]

Cp = np.matmul(Sp12,C)

def find_orth(O,vec):
    A = np.hstack((O, vec.reshape((O.shape[0],1))))
    b = np.zeros(O.shape[1] + 1)
    b[-1] = 1
    return lstsq(A.T, b)[0]

Cpnew = Cp[:,0]
Cpnew = Cpnew.reshape((Cpnew.shape[0],1))

for i in range(1,C.shape[1]):
    res = find_orth(Cpnew,Cp[:,i])
    res = res/np.linalg.norm(res)

    Cpnew = np.hstack((Cpnew,res.reshape((Cpnew.shape[0],1))))

C = np.matmul(Sm12,Cpnew)

/home/jfhlewyee/anaconda3/envs/pydonof/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


In [80]:
for i in range(C.shape[0]):
    print("------------------")
    for j in range(C.shape[0]):
        print(i,j,C[j][i])

------------------
0 0 -1.0007526590398925
0 1 0.007004106285950935
0 2 0.014946642394905695
0 3 0.00033909231413545846
0 4 5.9028014481644606e-06
0 5 0.004205042446625194
0 6 0.0002708883619920407
0 7 2.5777940464846853e-06
0 8 0.0010709696313503897
0 9 -0.0018626123963154902
0 10 -2.780064536433535e-17
0 11 -1.0810613458246898e-07
0 12 -0.0019327541568238498
0 13 5.521541458747815e-08
0 14 -0.0019766443235722723
0 15 0.000760342787310063
0 16 -0.001606460925968173
0 17 -1.5673097785644123e-07
0 18 -0.00015321997888803715
0 19 9.792084779693628e-06
0 20 0.0007513970844012656
0 21 -0.0016089907540805832
0 22 -1.5673097812034724e-07
0 23 0.00015299099523816395
0 24 9.559936351412335e-06
------------------
1 0 0.000989704888164374
1 1 0.1324330789429575
1 2 0.013992554370135718
1 3 -0.0003600566897758594
1 4 0.3546262427738014
1 5 -0.3345424647829638
1 6 -0.00018350394216833977
1 7 0.15729378908424882
1 8 -0.20512334967899856
1 9 0.005985471576479245
1 10 3.8216500580559327e-17
1 11 3.05

**Revisamos la ortonormalidad.** Asumiremos que los orbitales son ortonormales si $C^T SC$ difiere en menos de $10^{-6}$ respecto a la matriz identidad. Si los orbitales no son ortonormales, haremos hasta 3 intentos por ortonormalizarlos.

In [81]:
# Revisa ortonormalidad
orthonormality = True

CTSC = np.matmul(np.matmul(np.transpose(C),S),C)
ortho_deviation = np.abs(CTSC - np.identity(CTSC.shape[0]))

if (np.any(ortho_deviation > 10**-6)):
    orthonormality = False

if not orthonormality:
    print("Orthonormality violations {:d}, Maximum Violation {:f}".format((ortho_deviation > 10**-6).sum(),ortho_deviation.max()))        
else:
    print("No violations of the orthonormality")

Orthonormality violations 1, Maximum Violation 0.000006


Se cambian de fase los obitales moleculares tal que **en cada orbital sea positivo el coeficiente de mayor magnitud**

In [82]:
# Vuelve positivo el elemento más largo de cada MO
for j in range(C.shape[1]):
    
    #Obtiene el índice del coeficiente con mayor valor absoluto del MO
    idxmaxabsval = 0
    for i in range(nbf):
        if(abs(C[i][j])>abs(C[idxmaxabsval][j])):
            idxmaxabsval = i
    
    # Ajusta el signo del MO
    sign = np.sign(C[idxmaxabsval][j])
    C[0:nbf,j] = sign*C[0:nbf,j]

In [83]:
nv = ncwo*ndoc

gamma = np.zeros((nbf5))
for i in range(ndoc):
    gamma[i] = np.arccos(np.sqrt(2.0*0.999-1.0))
    for j in range(ncwo-1):
        ig = ndoc+(i)*(ncwo-1)+j
        gamma[ig] = np.arcsin(np.sqrt(1.0/(ncwo-j)))        

In [84]:
iloop = 0
itlim = 1
E_old = E
E_diff = 0
sumdiff_old = 0

print('{:^7} {:^7} {:^14} {:^14} {:^14} {:^14}'.format("Nitext","Nitint","Eelec","Etot","Ediff","maxdiff"))

for i_ext in range(1,1000):
    
    #orboptr
    convgdelag,E_old,E_diff,sumdiff_old,itlim,nzeros,fmiug0,C = orboptr(C,n,cj12,ck12,E_old,E_diff,sumdiff_old,i_ext,itlim,nzeros,fmiug0)     
    
    #occopt
    gamma,elag,n,cj12,ck12 = occoptr(gamma,False,convgdelag,elag,C,H)
        
    if(convgdelag):
        break

Nitext  Nitint      Eelec           Etot          Ediff         maxdiff    
     2     29   -79.15562988   -69.80101918     6.18301415     0.25209653
     3     29   -82.25880594   -72.90419523    -3.10317605     0.18329427
     4     29   -82.40971180   -73.05510110    -0.15090586     0.14381694
     5     29   -82.44938972   -73.09477901    -0.03967792     0.31442570
     6     29   -82.48418053   -73.12956982    -0.03479081     0.20186308
     7     29   -82.51525718   -73.16064648    -0.03107666     0.17010023
     8     29   -82.54683433   -73.19222363    -0.03157715     0.18677488
     9     29   -82.54431792   -73.18970722     0.00251641     0.24121725
    10     29   -82.58577571   -73.23116501    -0.04145779     0.34080393
    11     29   -82.62537251   -73.27076181    -0.03959680     0.20492105
    12     29   -82.67273248   -73.31812178    -0.04735997     0.19308621
    13     29   -82.69947669   -73.34486598    -0.02674420     0.26544006
    14     29   -82.72681727   -73.3

   113     29   -83.70170215   -74.34709145    -0.00306253     0.21767341
   114     29   -83.70471305   -74.35010234    -0.00301090     0.21744097
   115     29   -83.70772086   -74.35311015    -0.00300781     0.21739225
   116     29   -83.70806495   -74.35345424    -0.00034409     0.21722867
   117     29   -83.70836563   -74.35375493    -0.00030069     0.21714415
   118     29   -83.70866538   -74.35405467    -0.00029974     0.21709372
   119     29   -83.70896445   -74.35435374    -0.00029907     0.21705680
   120     29   -83.70926271   -74.35465201    -0.00029826     0.21700835
   121     29   -83.70956171   -74.35495101    -0.00029900     0.21698341
   122     29   -83.70985946   -74.35524876    -0.00029775     0.21697760
   123     29   -83.71015774   -74.35554703    -0.00029827     0.21696017
   124     29   -83.71045587   -74.35584517    -0.00029814     0.21694119
   125     29   -83.71075362   -74.35614292    -0.00029775     0.21692984
   126     29   -83.71105132   -74.356

   225     29   -84.33771526   -74.98310455    -0.00002158     0.22118463
   226     29   -84.33773683   -74.98312612    -0.00002157     0.22118744
   227     29   -84.35488730   -75.00027660    -0.01715047     0.22372813
   228     29   -84.37509512   -75.02048442    -0.02020782     0.22494303
   229     29   -84.39321901   -75.03860830    -0.01812388     0.22696824
   230     29   -84.41488119   -75.06027049    -0.02166218     0.22419451
   231     29   -84.43330192   -75.07869121    -0.01842073     0.22627979
   232     29   -84.45236220   -75.09775150    -0.01906028     0.22701510
   233     29   -84.46904979   -75.11443909    -0.01668759     0.22857109
   234     29   -84.48463837   -75.13002767    -0.01558858     0.22692806
   235     29   -84.49932902   -75.14471832    -0.01469065     0.22927642
   236     29   -84.51370410   -75.15909339    -0.01437507     0.23103891
   237     29   -84.52590108   -75.17129038    -0.01219698     0.22975903
   238     29   -84.53087225   -75.176

   337     29   -84.78422931   -75.42961861    -0.00013104     0.23348383
   338     29   -84.78436008   -75.42974937    -0.00013076     0.23350131
   339     29   -84.78449074   -75.42988003    -0.00013066     0.23352804
   340     29   -84.78462119   -75.43001048    -0.00013045     0.23354486
   341     29   -84.78475159   -75.43014088    -0.00013040     0.23356107
   342     29   -84.78476516   -75.43015446    -0.00001358     0.23356470
   343     29   -84.78477823   -75.43016752    -0.00001306     0.23357259
   344     29   -84.78479128   -75.43018058    -0.00001305     0.23357964
   345     29   -84.78480433   -75.43019363    -0.00001305     0.23358523
   346     29   -84.78481738   -75.43020668    -0.00001305     0.23358973
   347     29   -84.78483043   -75.43021972    -0.00001305     0.23359456
   348     29   -84.78484347   -75.43023277    -0.00001304     0.23359847
   349     29   -84.78485651   -75.43024581    -0.00001304     0.23360185
   350     29   -84.78486956   -75.430

   449     29   -85.09521341   -75.74060270    -0.00308949     0.16262766
   450     29   -85.09681502   -75.74220432    -0.00160161     0.16321574
   451     29   -85.09718555   -75.74257484    -0.00037053     0.16315171
   452     29   -85.09754172   -75.74293102    -0.00035618     0.16303048
   453     29   -85.09788436   -75.74327365    -0.00034263     0.16287111
   454     29   -85.09822677   -75.74361607    -0.00034242     0.16263285
   455     29   -85.09855195   -75.74394124    -0.00032517     0.16241111
   456     29   -85.09890419   -75.74429348    -0.00035224     0.16229770
   457     29   -85.09923059   -75.74461989    -0.00032641     0.16200862
   458     29   -85.09956965   -75.74495895    -0.00033906     0.16180486
   459     29   -85.09989251   -75.74528181    -0.00032286     0.16161128
   460     29   -85.09993609   -75.74532539    -0.00004358     0.16161341
   461     29   -85.09996910   -75.74535840    -0.00003301     0.16159646
   462     29   -85.10000190   -75.745

   561     29   -85.14318125   -75.78857055    -0.00000106     0.11235791
   562     29   -85.14318231   -75.78857161    -0.00000106     0.11235624
   563     29   -85.14318337   -75.78857267    -0.00000106     0.11235418
   564     29   -85.14318443   -75.78857373    -0.00000106     0.11235271
   565     29   -85.14318549   -75.78857479    -0.00000106     0.11235117
   566     29   -85.14318655   -75.78857585    -0.00000106     0.11234951
   567     29   -85.14318761   -75.78857691    -0.00000106     0.11234777
   568     29   -85.14318867   -75.78857796    -0.00000105     0.11234585
   569     29   -85.14330081   -75.78869010    -0.00011214     0.11104608
   570     29   -85.14405712   -75.78944641    -0.00075631     0.10898147
   571     29   -85.14526739   -75.79065669    -0.00121027     0.10731700
   572     29   -85.14611674   -75.79150603    -0.00084935     0.10524042
   573     29   -85.14808879   -75.79347809    -0.00197206     0.10404152
   574     29   -85.14719131   -75.792

   673     29   -85.17217980   -75.81756910    -0.00001512     0.01062875
   674     29   -85.17219487   -75.81758417    -0.00001507     0.00920430
   675     29   -85.17221963   -75.81760893    -0.00002476     0.00274678
   676     29   -85.17221335   -75.81760264     0.00000628     0.00964322
   677     29   -85.17223811   -75.81762740    -0.00002476     0.00243728
   678     29   -85.17223938   -75.81762867    -0.00000127     0.00244297
   679     29   -85.17224041   -75.81762971    -0.00000103     0.00244891
   680     29   -85.17224147   -75.81763076    -0.00000106     0.00245181
   681     29   -85.17224278   -75.81763208    -0.00000132     0.00244245
   682     29   -85.17224384   -75.81763314    -0.00000106     0.00245365
   683     29   -85.17224486   -75.81763416    -0.00000102     0.00245371
   684     29   -85.17224605   -75.81763534    -0.00000118     0.00245068
   685     29   -85.17224689   -75.81763618    -0.00000084     0.00245162
   686     29   -85.17224798   -75.817

   785     29   -85.17240033   -75.81778963     0.00048662     0.00656204
   786     29   -85.17240182   -75.81779112    -0.00000149     0.02106092
   787     29   -85.17218667   -75.81757597     0.00021515     0.01860459
   788     29   -85.17198545   -75.81737475     0.00020122     0.03427222
   789     29   -85.17252196   -75.81791126    -0.00053651     0.01154657
   790     29   -85.17085296   -75.81624225     0.00166901     0.09439251
   791     29   -85.17217838   -75.81756767    -0.00132542     0.01475854
   792     29   -85.17145476   -75.81684406     0.00072362     0.07029957
   793     29   -85.17204033   -75.81742963    -0.00058557     0.02144826
   794     29   -85.17211949   -75.81750879    -0.00007916     0.01428193
   795     29   -85.17066438   -75.81605368     0.00145511     0.08928770
   796     29   -85.17289816   -75.81828746    -0.00223378     0.00045952
   797     29   -85.17289493   -75.81828423     0.00000323     0.00385916
   798     29   -85.17288621   -75.818

In [46]:
for i in range(C.shape[0]):
    print("------------------")
    for j in range(C.shape[0]):
        print(i,j,C[j][i])

------------------
0 0 1.001018936809926
0 1 -0.006540505916255013
0 2 -0.010207436079791224
0 3 -1.5869957600109668e-05
0 4 1.6044971214407628e-07
0 5 -0.005186165080446833
0 6 -1.259362423157878e-05
0 7 8.627877077471466e-07
0 8 -0.0032348215947928943
0 9 -0.0013352839167036306
0 10 6.530247663551321e-07
0 11 1.875298446837384e-07
0 12 -0.0011186617251804116
0 13 -1.4594191862488654e-06
0 14 -0.0009855288522043035
0 15 -0.0017628260921073162
0 16 0.00034249057551875574
0 17 -9.227863981135689e-07
0 18 0.000835030702430377
0 19 -0.00077798824494242
0 20 -0.0017612917652864862
0 21 0.00034281851626775673
0 22 -7.2884314275286e-07
0 23 -0.0008343816607443624
0 24 -0.0007774248811973167
------------------
1 0 -0.006667513689134486
1 1 -0.2968324036207758
1 2 -0.36346179414276364
1 3 0.44444910984111197
1 4 -3.1176818375619795e-05
1 5 -0.23092921166580008
1 6 0.3516249586494426
1 7 -4.427353380351394e-05
1 8 -0.19148323259626313
1 9 -0.002038420920134705
1 10 -2.3020767228585967e-05
1 11 